In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [152]:
# Get a page
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = 'https://www.congress.gov/congressional-record/2018/02/12/senate-section'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [153]:
# Extract links

In [154]:
table_rows = soup.find('table', class_='item_table').find_all('tr')
a_elts = [tr.find('a') for tr in table_rows]
a_elts = [a for a in a_elts if a is not None]
links = {a.string: 'https://www.congress.gov' + a.get('href') for a in a_elts}

In [155]:
l = links['IMMIGRATION']

In [156]:
item_page = requests.get(l, headers=headers)
soup = BeautifulSoup(item_page.text, 'html.parser')
text = soup.find('pre').get_text()
print(text)

        
[Congressional Record Volume 164, Number 27 (Monday, February 12, 2018)]
[Pages S855-S856]
From the Congressional Record Online through the Government Publishing Office [www.gpo.gov]




                              IMMIGRATION

  Mr. McCONNELL. Now on another matter, Mr. President, as I have 
repeatedly stated, now that there is an agreement on long-term 
government funding, the Senate will proceed to a fair debate over the 
DACA issue, border security, and other matters pertaining to the 
subject of immigration. This process begins in just a few hours. We 
will vote to proceed to an unrelated, neutral House-passed bill that 
will serve as the vehicle for any legislation that succeeds here in the 
Senate. As I have repeated many times, I will ensure that a fair 
amendment process gives Senators the opportunity to submit proposals 
for consideration and votes. I hope this body can seize this 
opportunity and deliver real progress toward securing our border, 
reforming aspects

In [160]:
tp = text
def header_transform(match):
    return f'### {match.group(1)} ###'

# Remove page break indicators, e.g. [[Page S856]]
tp = re.sub('\n\n\[\[.*\]\]\n\n', ' ', tp)

# Transform the header, replacing '[foo]' with '### foo ###' and removing an unhelpful line
tp = re.sub('^\[(.*)\]$', header_transform, tp, flags=re.MULTILINE)
tp = re.sub('From the Congressional Record Online through the Government Publishing Office \[www.gpo.gov\]', '', tp)

# Format title like header
tp = re.sub(' *IMMIGRATION', '### IMMIGRATION ###', tp)

# Remove footer line
tp = re.sub(' *____________________', '', tp)

# Collapse multiple line breaks
tp = re.sub('[\n]{2,}', '\n', tp)

# Remove all line breaks in main body
tp = re.sub('  ', '', tp)
tp = re.sub(' \n', ' ', tp)
tp = re.sub('\.\n', '. ', tp)

print(tp)


### Congressional Record Volume 164, Number 27 (Monday, February 12, 2018) ###
### Pages S855-S856 ###
### IMMIGRATION ###
Mr. McCONNELL. Now on another matter, Mr. President, as I have repeatedly stated, now that there is an agreement on long-term government funding, the Senate will proceed to a fair debate over the DACA issue, border security, and other matters pertaining to the subject of immigration. This process begins in just a few hours. We will vote to proceed to an unrelated, neutral House-passed bill that will serve as the vehicle for any legislation that succeeds here in the Senate. As I have repeated many times, I will ensure that a fair amendment process gives Senators the opportunity to submit proposals for consideration and votes. I hope this body can seize this opportunity and deliver real progress toward securing our border, reforming aspects of our immigration policy, and achieving a resolution for individuals who were brought to our country illegally when they were 